In [1]:
import importlib
import full_model
import numpy as np

In [2]:
full_model.eng = full_model.start_matlab()
show_plot = False


Starting MATLAB engine...
-------------------------
Elapsed: 2.71 sec


## Try various Hyperparameter tuning methods
* [Scikit-Optimize](https://scikit-optimize.github.io/notebooks/hyperparameter-optimization.html)

In [3]:
from skopt.utils import use_named_args

from skopt.space import Real, Integer
space  = [Real(0.1, 1, "log-uniform", name='lr_pretrain'),
          Real(0.1, 1, "log-uniform", name='lr'),
          Real(0.1, 1, "log-uniform", name='alpha1'),
          Integer(0, 1, name='maxIter1'),
          Real(0.1, 1, "log-uniform", name='alpha2'),
          Integer(0, 1, name='maxIter2')]

@use_named_args(space)
def objective(**params):
    global ep_prtr_, ep_, seed_, verb_
    try:
        return 1-full_model.run_model(images_norm, labels, seed_, ep_prtr_, ep_, **params, verbose=verb_)[0]
    except Exception as ex:
        if(type(ex) == KeyError):
            raise ex
        print("Caught a " + type(ex).__name__ + ". Returning 1.0")
        return 1

In [4]:
from skopt.plots import plot_convergence
import matplotlib.pyplot as plt

def opt_stats(res, start_time=None):
    print("------------------")
    for i in range(10, len(res.func_vals)+1, 10):
        print("{0:d}: {1:.4f}".format(i, min(res.func_vals[0:i])))
    print("Best score: {0:.4f}".format(res.fun))
    print("Best parameters: {0:}".format(res.x))
    if(start_time is not None):
        print("Total time elapsed: {0:.2f} sec\n".format(time.time()-start_time))

def opt_plot(res):
    global show_plot
    plot = plot_convergence(res, yscale="log")
    if(show_plot):
        plt.show(plot)
    return plot

In [5]:
from skopt import gp_minimize, dummy_minimize, forest_minimize, gbrt_minimize
import time

def optimize(function, iterations, epochs_pretrain, epochs, random_seed, verb_model=False, verb=True):
    start_time = time.time()
    
    # global b/c I couldn't find a better way to directly pass these to the objective function
    global ep_prtr_, ep_, seed_, verb_
    ep_prtr_ = epochs_pretrain
    ep_ = epochs
    seed_ = random_seed
    verb_ = verb_model
    
    # kwargs b/c dummy_minimize can not take n_jobs
    params = {"n_calls":iterations, "random_state":random_seed, "verbose":verb}
    if(function != dummy_minimize):
        params["n_jobs"] = -1
    result = function(objective, space, **params)
    if(verb):
        opt_plot(result)
        print()
    opt_stats(result, start_time)
    return result

In [6]:
from copy import deepcopy
from skopt.learning import GaussianProcessRegressor
from skopt.optimizer import base_minimize
from sklearn.utils import check_random_state
import warnings

def func_new(params):
    global func_, xs_, ys_
    if(len(xs_) > 0):
        y = ys_.pop(0)
        if(params != xs_.pop(0)):
            warnings.warn("Deviated from expected value, re-evaluating", RuntimeWarning)
        else:
            return y
    return func_(params)

def reload(result, addtl_iters, epochs_pretrain, epochs, random_seed, verb_model=False, verb=True):
    start_time = time.time()
    
    # since objective relies on global variables, set them again
    global ep_prtr_, ep_, seed_, verb_
    ep_prtr_ = epochs_pretrain
    ep_ = epochs
    seed_ = random_seed
    verb_ = verb_model
    
    # retrieve optimization call's arguments
    args = deepcopy(result.specs['args'])
    args['n_calls'] += addtl_iters
    args['verbose'] = verb
    
    # global b/c I couldn't find a better way to pass
    global func_, xs_, ys_
    func_ = args['func']
    xs_ = list(result.x_iters)
    ys_ = list(result.func_vals)
    args['func'] = func_new
    
    # recover initial random_state
    if(isinstance(args['random_state'], np.random.RandomState)):
        args['random_state'] = check_random_state(random_seed)
        # if gp_minimize
        if(isinstance(result.specs['args']['base_estimator'], GaussianProcessRegressor)):
            args['random_state'].randint(0, np.iinfo(np.int32).max)
    
    # run the optimization
    result_new = base_minimize(**args)
    
    # change the function back, to reload multiple times
    result_new.specs['args']['func'] = func_
    
    if(verb):
        opt_plot(result_new)
        print()
    opt_stats(result_new, start_time)
    return result_new

In [ ]:
import os
from skopt import dump

scenario = 3
dataset = "Coil20.mat"
iters = 50
ep_prtr = 1001
ep = 251
space  = [Real(10**-4, 10**0, "log-uniform", name='lr_pretrain'),
          Real(10**-5, 10**-1, "log-uniform", name='lr'),
          Real(10**-1, 10**3, "log-uniform", name='alpha1'),
          Integer(10, 200, name='maxIter1'),
          Real(10**-1, 10**3, "log-uniform", name='alpha2'),
          Integer(10, 200, name='maxIter2')]
data_loaded = full_model.loadmat("./saved/processed/" + dataset)
images_norm = data_loaded['X']
labels = data_loaded['Y'].reshape(-1)
if(not os.path.isdir("optims/scenario" + str(scenario))):
    os.mkdir("optims/scenario" + str(scenario))

#RUN
for seed in range(5):
    print("Seed: " + str(seed))
    
    print("GP:")
    gp = optimize(gp_minimize, iters, ep_prtr, ep, seed, verb=False)
    dump(gp, "optims/scenario" + str(scenario) + "/gp_" + str(seed) + "_" + str(iters) + ".opt")
    
    print("DUMMY:")
    dummy = optimize(dummy_minimize, iters, ep_prtr, ep, seed, verb=False)
    dump(dummy, "optims/scenario" + str(scenario) + "/dummy_" + str(seed) + "_" + str(iters) + ".opt")
    
    print("FOREST:")
    forest = optimize(forest_minimize, iters, ep_prtr, ep, seed, verb=False)
    dump(forest, "optims/scenario" + str(scenario) + "/forest_" + str(seed) + "_" + str(iters) + ".opt")
    
    print("GBRT:")
    gbrt = optimize(gbrt_minimize, iters, ep_prtr, ep, seed, verb=False)
    dump(gbrt, "optims/scenario" + str(scenario) + "/gbrt_" + str(seed) + "_" + str(iters) + ".opt")

In [ ]:
import os
from skopt import dump, load

scenario = 3
dataset = "Coil20.mat"
iters = 50
addtl_iters = 50
ep_prtr = 1001
ep = 251
data_loaded = full_model.loadmat("./saved/processed/" + dataset)
images_norm = data_loaded['X']
labels = data_loaded['Y'].reshape(-1)

#RE-RUN
for seed in range(5):
    print("Seed: " + str(seed))
    
    print("GP:")
    gp_loaded = load("optims/scenario" + str(scenario) + "/gp_" + str(seed) + "_" + str(iters) + ".opt")
    gp = reload(gp_loaded, addtl_iters, ep_prtr, ep, seed, verb=False)
    dump(gp, "optims/scenario" + str(scenario) + "/gp_" + str(seed) + "_" + str(iters+addtl_iters) + ".opt")
    
    print("DUMMY:")
    dummy_loaded = load("optims/scenario" + str(scenario) + "/dummy_" + str(seed) + "_" + str(iters) + ".opt")
    dummy = reload(dummy_loaded, addtl_iters, ep_prtr, ep, seed, verb=False)
    dump(dummy, "optims/scenario" + str(scenario) + "/dummy_" + str(seed) + "_" + str(iters+addtl_iters) + ".opt")
    
    print("FOREST:")
    forest_loaded = load("optims/scenario" + str(scenario) + "/forest_" + str(seed) + "_" + str(iters) + ".opt")
    forest = reload(forest_loaded, addtl_iters, ep_prtr, ep, seed, verb=False)
    dump(forest, "optims/scenario" + str(scenario) + "/forest_" + str(seed) + "_" + str(iters+addtl_iters) + ".opt")
    
    print("GBRT:")
    gbrt_loaded = load("optims/scenario" + str(scenario) + "/gbrt_" + str(seed) + "_" + str(iters) + ".opt")
    gbrt = reload(gbrt_loaded, addtl_iters, ep_prtr, ep, seed, verb=False)
    dump(gbrt, "optims/scenario" + str(scenario) + "/gbrt_" + str(seed) + "_" + str(iters+addtl_iters) + ".opt")

Seed: 0
GP:
------------------
10: 0.4340
20: 0.3278
30: 0.3215
40: 0.3215
50: 0.3215
60: 0.3215
70: 0.3174
80: 0.3174
90: 0.2979
100: 0.2896
Best score: 0.2896
Best parameters: [0.010079462144198303, 0.0008881240820334886, 1000.0, 10, 1000.0, 10]
Total time elapsed: 3438.71 sec

DUMMY:
------------------
10: 0.4264
20: 0.4264
30: 0.4083
40: 0.4083
50: 0.4083
60: 0.3847
70: 0.3625
80: 0.3625
90: 0.3389
100: 0.3389
Best score: 0.3389
Best parameters: [0.002706636290078833, 0.00034749589148156887, 464.4169664210571, 97, 336.41925856077427, 184]
Total time elapsed: 4088.44 sec

FOREST:
Caught a MatlabExecutionError. Returning 1.0
Caught a MatlabExecutionError. Returning 1.0
------------------
10: 0.4264
20: 0.3236
30: 0.3236
40: 0.2875
50: 0.2875
60: 0.2875
70: 0.2875
80: 0.2875
90: 0.2875
100: 0.2875
Best score: 0.2875
Best parameters: [0.27132766378910406, 0.001380381361838373, 0.2518454319618381, 74, 410.1861235669267, 62]
Total time elapsed: 3444.12 sec

GBRT:
------------------
10: 0

In [ ]:
scenario_load = 0
seed_load = 1
iters_load = 50

In [ ]:
from skopt import load, dump

scenario = 1
dataset = "yaleB.mat"
iters = 50
addtl_iters = 50
ep_prtr = 201
ep = 101
space  = [Real(10**-2, 10**0, "log-uniform", name='lr_pretrain'),
          Real(10**-3, 10**-1, "log-uniform", name='lr'),
          Real(10**0, 10**2, "log-uniform", name='alpha1'),
          Integer(10, 32, name='maxIter1'),
          Real(10**0, 10**2, "log-uniform", name='alpha2'),
          Integer(10, 32, name='maxIter2')]
data_loaded = full_model.loadmat("./saved/processed/" + dataset)
images_norm = data_loaded['X']
labels = data_loaded['Y'].reshape(-1)

dummy_loaded = load("optims/scenario" + str(scenario) + "/dummy_" + str(0) + "_" + str(iters) + ".opt")

In [ ]:
dummy = reload(dummy_loaded, 3, ep_prtr, ep, 0, verb=True, verb_model=True)

In [ ]:
dummy_loaded.func_vals

In [ ]:
dummy = optimize(dummy_minimize, 4, ep_prtr, ep, 0, verb=True, verb_model=True)

In [40]:
from skopt import load
gp_loaded = load("optims/scenario" + str(scenario_load) + "/gp_" + str(seed_load) + "_" + str(iters_load) + ".opt")
dummy_loaded = load("optims/scenario" + str(scenario_load) + "/dummy_" + str(seed_load) + "_" + str(iters_load) + ".opt")
forest_loaded = load("optims/scenario" + str(scenario_load) + "/forest_" + str(seed_load) + "_" + str(iters_load) + ".opt")
gbrt_loaded = load("optims/scenario" + str(scenario_load) + "/gbrt_" + str(seed_load) + "_" + str(iters_load) + ".opt")

In [44]:
result_loaded = forest_loaded
for i in range(10, iters_load+1, 10):
    print("\t{0:d}: {1:.4f}".format(i, min(result_loaded.func_vals[0:i])))

	10: 0.0634
	20: 0.0017
	30: 0.0013
	40: 0.0013
	50: 0.0013


In [46]:
forest_loaded.x

[0.07810429304974488,
 0.0015891561915202352,
 21.38388646813923,
 7,
 10.283602902208075,
 30]

In [10]:
tests = {'fun', 'space', 'x', 'func_vals', 'x_iters'}
def test(val1, val2):
    assert np.alltrue([np.alltrue(val1[key] == val2[key]) for key in tests]), "TESTS FAILED"
    print("ALL TESTS PASSED!")

In [ ]:
used_func = forest_minimize
seed = 1234
res1 = optimize(used_func, 12, 1, 1, seed)
res1_copy = deepcopy(res1)
res12 = reload(res1, 4, seed)
res23 = reload(res12, 4, seed)
res13 = reload(res1, 8, seed)
res3 = optimize(used_func, 20, 1, 1, seed)

from skopt import dump, load
dump(res1, "temp.txt")
res1_loaded = load("temp.txt")
res12_loaded = reload(res1_loaded, 4, seed)

In [97]:
test(res13, res3)
test(res23, res3)
test(res1, res1_loaded)
test(res12, res12_loaded)
test(res1_copy, res1)

ALL TESTS PASSED!
ALL TESTS PASSED!
ALL TESTS PASSED!
ALL TESTS PASSED!
ALL TESTS PASSED!
